## Initial ingest

### Context
Ingest manually collected data on grants and grantees into a Neo4J database 
to establish a basis for future enhancements through the development of ingestion pipelines 
for on/off-chain data.


Very sloppy but doing what I can on short notice :)

In [9]:
import os 
import pandas as pd
import time
import neo4j
from neo4j import GraphDatabase

from dotenv import load_dotenv

load_dotenv()

True

In [10]:
## Neo4J auth
## Free Neo4J instances are available via Neo4J aura - https://neo4j.com/cloud/aura-free/ 
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_PASS = os.getenv('NEO4J_PASS')
neo4j_client = GraphDatabase.driver(NEO4J_URI, auth=('neo4j', NEO4J_PASS))
retrieval = "InitialIngest"

def execute_query(query, params=None, neo4j=neo4j):
    with neo4j_client.session() as session:
        result = session.run(query, parameters=params)
        # Convert the result to a DataFrame
        records = [record.data() for record in result]
        return pd.DataFrame(records)


# `grantees.csv`
Ingest `Grantees` who received Arbitrum Foundation funds, directly or through matching programs.

## Grantee
Purposefully minimal -- identifiers are separate nodes to facilitate reconciliation for future ingests

`(grantee:Entity:Grantee {name})`

In [11]:
grantees_filname = 'grantees_20240303.csv'
grantees_df = pd.read_csv(f'inputs/{grantees_filname}')
print(f"There are {len(grantees_df)} grantees to ingest")

There are 152 grantees to ingest


In [12]:
## iterate through df and create entities
### todo - use LOAD JSON | LOAD CSV for efficiency
    

def create_grantees(df):
    total = len(df)
    counter = 0 
    for index, row in df.iterrows():
        counter += 1
        print(f"Ingesting grantee {str(counter)} of {total}...")

        params = {
            'name': row['Name']
        }
        
        query = """
        MERGE (entity:Grantee:Entity {name: $name})
        """
        execute_query(query, params)

create_grantees(grantees_df)


Ingesting grantee 1 of 152...
Ingesting grantee 2 of 152...
Ingesting grantee 3 of 152...
Ingesting grantee 4 of 152...
Ingesting grantee 5 of 152...
Ingesting grantee 6 of 152...
Ingesting grantee 7 of 152...
Ingesting grantee 8 of 152...
Ingesting grantee 9 of 152...
Ingesting grantee 10 of 152...
Ingesting grantee 11 of 152...
Ingesting grantee 12 of 152...
Ingesting grantee 13 of 152...
Ingesting grantee 14 of 152...
Ingesting grantee 15 of 152...
Ingesting grantee 16 of 152...
Ingesting grantee 17 of 152...
Ingesting grantee 18 of 152...
Ingesting grantee 19 of 152...
Ingesting grantee 20 of 152...
Ingesting grantee 21 of 152...
Ingesting grantee 22 of 152...
Ingesting grantee 23 of 152...
Ingesting grantee 24 of 152...
Ingesting grantee 25 of 152...
Ingesting grantee 26 of 152...
Ingesting grantee 27 of 152...
Ingesting grantee 28 of 152...
Ingesting grantee 29 of 152...
Ingesting grantee 30 of 152...
Ingesting grantee 31 of 152...
Ingesting grantee 32 of 152...
Ingesting grantee

In [13]:
## verify grantees created
execute_query("""MATCH (grantee:Grantee) RETURN COUNT(*)""")

,COUNT(*)
0,152


## Grantee identifiers

Identifiers are broken out as separate nodes because they can or maintain their own relationships (i.e. a GitHub account contributes to Repos, Twitter Account posts Tweets)

### Todos

Set `.url` as property on `:Account` nodes where applicable.


**Twitter**

`(grantee:Grantee)-[r:ACCOUNT]->(twitter:Twitter:Account {handle})`


**Github**

`(grantee:Grantee)-[r:ACCOUNT]->(github:Website:Account {url})`

**Wallet**

`(grantee:Grantee)-[r:ACCOUNT {source}]->(wallet:Wallet {address})`

*(note: add label for multisigs)*

*(note: add sources when ingest `grants.csv`)*

So on and so forth...

In [14]:
## twitter

counter = 0 
grantees_df_twitter = grantees_df[['Name', 'Twitter']] 
grantees_df_twitter = grantees_df_twitter.dropna(subset=['Twitter'])
total = len(grantees_df_twitter)

print(f"Ingesting {str(total)} twitter accounts...")
counter = 0 

for index, row in grantees_df_twitter.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} of {str(total)} records...")
    params = {
        'handle': row['Twitter'],
        'name': row['Name']
    }
    
    query = """
    merge (twitter:Twitter:Account {handle: $handle}) 
    with twitter
    match (grantee:Grantee {name: $name}) 
    with grantee, twitter
    merge (grantee)-[r:ACCOUNT]->(twitter)
    """
    execute_query(query, params)

execute_query("""match (grantee:Grantee)-[r]-(twitter:Twitter) return count(distinct(r)) as linked_twitters""")
    
    

Ingesting 116 twitter accounts...
Ingesting 1 of 116 records...
Ingesting 2 of 116 records...
Ingesting 3 of 116 records...
Ingesting 4 of 116 records...
Ingesting 5 of 116 records...
Ingesting 6 of 116 records...
Ingesting 7 of 116 records...
Ingesting 8 of 116 records...
Ingesting 9 of 116 records...
Ingesting 10 of 116 records...
Ingesting 11 of 116 records...
Ingesting 12 of 116 records...
Ingesting 13 of 116 records...
Ingesting 14 of 116 records...
Ingesting 15 of 116 records...
Ingesting 16 of 116 records...
Ingesting 17 of 116 records...
Ingesting 18 of 116 records...
Ingesting 19 of 116 records...
Ingesting 20 of 116 records...
Ingesting 21 of 116 records...
Ingesting 22 of 116 records...
Ingesting 23 of 116 records...
Ingesting 24 of 116 records...
Ingesting 25 of 116 records...
Ingesting 26 of 116 records...
Ingesting 27 of 116 records...
Ingesting 28 of 116 records...
Ingesting 29 of 116 records...
Ingesting 30 of 116 records...
Ingesting 31 of 116 records...
Ingesting 32 o

,linked_twitters
0,116


In [15]:
## ingest websites

## twitter

counter = 0 
grantees_df_websites = grantees_df[['Name', 'Website']] 
grantees_df_websites = grantees_df_websites.dropna(subset=['Website'])
total = len(grantees_df_websites)

print(f"Ingesting {str(total)} websites...")
counter = 0 

for index, row in grantees_df_websites.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} of {str(total)} records...")
    params = {
        'url': row['Website'],
        'name': row['Name']
    }
    
    query = """
    merge (website:Website:Account {url: $url}) 
    with website
    match (grantee:Grantee {name: $name}) 
    with grantee, website
    merge (grantee)-[r:ACCOUNT]->(website)
    """
    execute_query(query, params)

execute_query("""match (grantee:Grantee)-[r]-(website:Website) return count(distinct(r)) as linked_websites""")
    
    

Ingesting 121 websites...
Ingesting 1 of 121 records...
Ingesting 2 of 121 records...
Ingesting 3 of 121 records...
Ingesting 4 of 121 records...
Ingesting 5 of 121 records...
Ingesting 6 of 121 records...
Ingesting 7 of 121 records...
Ingesting 8 of 121 records...
Ingesting 9 of 121 records...
Ingesting 10 of 121 records...
Ingesting 11 of 121 records...
Ingesting 12 of 121 records...
Ingesting 13 of 121 records...
Ingesting 14 of 121 records...
Ingesting 15 of 121 records...
Ingesting 16 of 121 records...
Ingesting 17 of 121 records...
Ingesting 18 of 121 records...
Ingesting 19 of 121 records...
Ingesting 20 of 121 records...
Ingesting 21 of 121 records...
Ingesting 22 of 121 records...
Ingesting 23 of 121 records...
Ingesting 24 of 121 records...
Ingesting 25 of 121 records...
Ingesting 26 of 121 records...
Ingesting 27 of 121 records...
Ingesting 28 of 121 records...
Ingesting 29 of 121 records...
Ingesting 30 of 121 records...
Ingesting 31 of 121 records...
Ingesting 32 of 121 re

,linked_websites
0,121


In [16]:
## ingest Githubs

## twitter

counter = 0 
grantees_df_githubs = grantees_df[['Name', 'Github']] 
grantees_df_githubs = grantees_df_githubs.dropna(subset=['Github'])
total = len(grantees_df_githubs)

print(f"Ingesting {str(total)} github accounts...")
counter = 0 

for index, row in grantees_df_githubs.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} of {str(total)} records...")
    params = {
        'handle': row['Github'],
        'name': row['Name']
    }
    
    query = """
    merge (github:Github:Account {handle: $handle}) 
    with github
    match (grantee:Grantee {name: $name}) 
    with grantee, github
    merge (grantee)-[r:ACCOUNT]->(github)
    """
    execute_query(query, params)

execute_query("""match (grantee:Grantee)-[r]-(github:Github) return count(distinct(r)) as linked_githubs""")
    
    

Ingesting 61 github accounts...
Ingesting 1 of 61 records...
Ingesting 2 of 61 records...
Ingesting 3 of 61 records...
Ingesting 4 of 61 records...
Ingesting 5 of 61 records...
Ingesting 6 of 61 records...
Ingesting 7 of 61 records...
Ingesting 8 of 61 records...
Ingesting 9 of 61 records...
Ingesting 10 of 61 records...
Ingesting 11 of 61 records...
Ingesting 12 of 61 records...
Ingesting 13 of 61 records...
Ingesting 14 of 61 records...
Ingesting 15 of 61 records...
Ingesting 16 of 61 records...
Ingesting 17 of 61 records...
Ingesting 18 of 61 records...
Ingesting 19 of 61 records...
Ingesting 20 of 61 records...
Ingesting 21 of 61 records...
Ingesting 22 of 61 records...
Ingesting 23 of 61 records...
Ingesting 24 of 61 records...
Ingesting 25 of 61 records...
Ingesting 26 of 61 records...
Ingesting 27 of 61 records...
Ingesting 28 of 61 records...
Ingesting 29 of 61 records...
Ingesting 30 of 61 records...
Ingesting 31 of 61 records...
Ingesting 32 of 61 records...
Ingesting 33 of 6

,linked_githubs
0,61


In [17]:
## ingest Dune

## twitter

counter = 0 
grantees_df_dunes = grantees_df[['Name', 'Dune']] 
grantees_df_dunes = grantees_df_dunes.dropna(subset=['Dune'])
total = len(grantees_df_dunes)

print(f"Ingesting {str(total)} dune accounts...")
counter = 0 

for index, row in grantees_df_dunes.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} of {str(total)} records...")
    params = {
        'handle': row['Dune'],
        'name': row['Name']
    }
    
    query = """
    merge (dune:Dune:Account {handle: $handle}) 
    with dune
    match (grantee:Grantee {name: $name}) 
    merge (grantee)-[r:ACCOUNT]->(dune)
    """
    execute_query(query, params)

execute_query("""match (grantee:Grantee)-[r]-(dune:Dune) return count(distinct(r)) as linked_dunes""")
    
    

Ingesting 4 dune accounts...
Ingesting 1 of 4 records...
Ingesting 2 of 4 records...
Ingesting 3 of 4 records...
Ingesting 4 of 4 records...


,linked_dunes
0,4


In [18]:
## ingest wallets

counter = 0 
grantees_df_wallets = grantees_df[['Name', 'Funding Wallet Address']] 
grantees_df_wallets = grantees_df_wallets.dropna(subset=['Funding Wallet Address'])
total = len(grantees_df_wallets)

print(f"Ingesting {str(total)} wallets...")
counter = 0 

for index, row in grantees_df_wallets.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} of {str(total)} records...")
    params = {
        'address': row['Funding Wallet Address'],
        'name': row['Name']
    }
    
    query = """
    merge (wallet:Wallet:Account {address: tolower($address)}) 
    with wallet
    match (grantee:Grantee {name: $name}) 
    merge (grantee)-[r:ACCOUNT]->(wallet)
    """
    execute_query(query, params)
    time.sleep(.5)

execute_query("""match (grantee:Grantee)-[r]-(wallet:Wallet) return count(distinct(r)) as linked_wallets""")
    
    

Ingesting 114 wallets...
Ingesting 1 of 114 records...
Ingesting 2 of 114 records...
Ingesting 3 of 114 records...
Ingesting 4 of 114 records...
Ingesting 5 of 114 records...
Ingesting 6 of 114 records...
Ingesting 7 of 114 records...
Ingesting 8 of 114 records...
Ingesting 9 of 114 records...
Ingesting 10 of 114 records...
Ingesting 11 of 114 records...
Ingesting 12 of 114 records...
Ingesting 13 of 114 records...
Ingesting 14 of 114 records...
Ingesting 15 of 114 records...
Ingesting 16 of 114 records...
Ingesting 17 of 114 records...
Ingesting 18 of 114 records...
Ingesting 19 of 114 records...
Ingesting 20 of 114 records...
Ingesting 21 of 114 records...
Ingesting 22 of 114 records...
Ingesting 23 of 114 records...
Ingesting 24 of 114 records...
Ingesting 25 of 114 records...
Ingesting 26 of 114 records...
Ingesting 27 of 114 records...
Ingesting 28 of 114 records...
Ingesting 29 of 114 records...
Ingesting 30 of 114 records...
Ingesting 31 of 114 records...
Ingesting 32 of 114 rec

,linked_wallets
0,114


## Ingest other grantees

`other_grantees_initial.csv` has Arbitrum ecosystem projects which I haven't collected structured grant data for yet
(i.e. they've received grants but I don't have the grant metadata structured yet)

In [19]:
other_grantees_filname = 'inputs/other_grantees_initial.csv'
other_grantees_df = pd.read_csv(other_grantees_filname)

print(f"There are {len(other_grantees_df)} records to ingest")

other_grantees_df.head(3)

There are 22 records to ingest


,name,website,twitter,github,source,platform
0,ArbitrumNews DAO,https://link3.to/arbitrumnewsdao,ArbitrumNewsDAO,ArbitrumNewsDAO,https://checker.gitcoin.co/public/project/show...,Gitcoin
1,Burns,https://forum.arbitrum.foundation/t/rfp-abitru...,mitchburns_,burns-unit,https://checker.gitcoin.co/public/project/show...,Gitcoin
2,Raho,raho.me/stip,raholloway,raho1,https://checker.gitcoin.co/public/project/show...,Gitcoin


In [20]:
## ingest wallets

counter = 0 
total = len(other_grantees_df)

for index, row in other_grantees_df.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} of {str(total)} records...")
    params = {
        'name': row['name']
    }
    
    query = """
    merge (grantee:Grantee:Entity {name: $name}) 
    """
    execute_query(query, params)
    time.sleep(.5)

execute_query("""match (grantee:Grantee) return count(grantee) as other_grantees""")
    
    

Ingesting 1 of 22 records...
Ingesting 2 of 22 records...
Ingesting 3 of 22 records...
Ingesting 4 of 22 records...
Ingesting 5 of 22 records...
Ingesting 6 of 22 records...
Ingesting 7 of 22 records...
Ingesting 8 of 22 records...
Ingesting 9 of 22 records...
Ingesting 10 of 22 records...
Ingesting 11 of 22 records...
Ingesting 12 of 22 records...
Ingesting 13 of 22 records...
Ingesting 14 of 22 records...
Ingesting 15 of 22 records...
Ingesting 16 of 22 records...
Ingesting 17 of 22 records...
Ingesting 18 of 22 records...
Ingesting 19 of 22 records...
Ingesting 20 of 22 records...
Ingesting 21 of 22 records...
Ingesting 22 of 22 records...


,other_grantees
0,165


In [21]:
## ingest other grantee metadata

other_twitter = other_grantees_df[['name', 'twitter']]
other_twitter = other_twitter.dropna(subset=['twitter'])

for index, row in other_twitter.iterrows():
    url = "https://twitter.com/" + row['twitter'] 
    params = {
        'handle': row['twitter'],
        'url': url,
        'name': row['name']
    }
    query = """
        merge (twitter:Account:Twitter {handle: $handle, url: $url})
        with twitter
        match (grantee:Grantee:Entity {name: $name}) 
        with twitter, grantee 
        merge (grantee)-[r:ACCOUNT]->(twitter)
        """
    execute_query(query, params)
    time.sleep(.5)


In [22]:
## ingest other grantee metadata

other_github = other_grantees_df[['name', 'github']]
other_github = other_github.dropna(subset=['github'])

for index, row in other_github.iterrows():
    url = "https://github.com.com/" + row['github'] 
    params = {
        'handle': row['github'],
        'url': url,
        'name': row['name']
    }
    query = """
        merge (github:Account:Github {handle: $handle, url: $url})
        with github
        match (grantee:Grantee:Entity {name: $name}) 
        with github, grantee 
        merge (grantee)-[r:ACCOUNT]->(github)
        """
    execute_query(query, params)
    time.sleep(.1)


In [23]:
## ingest other grantee metadata

other_github = other_grantees_df[['name', 'github']]
other_github = other_github.dropna(subset=['github'])

for index, row in other_github.iterrows():
    url = "https://github.com.com/" + row['github'] 
    params = {
        'handle': row['github'],
        'url': url,
        'name': row['name']
    }
    query = """
        merge (github:Account:Github {handle: $handle, url: $url})
        with github
        match (grantee:Grantee:Entity {name: $name}) 
        with github, grantee 
        merge (grantee)-[r:ACCOUNT]->(github)
        """
    execute_query(query, params)
    time.sleep(.1)


In [24]:
## temporarily set `.source` on other grantees to establish Arbitrum ecosystem connection
### i.e. `https://checker.gitcoin.co/public/project/show/francescoweb3-creating-educational-content-on-arbitrum`

## ingest other grantee metadata

## ingest wallets

counter = 0 
total = len(other_grantees_df)

for index, row in other_grantees_df.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} of {str(total)} records...")
    params = {
        'name': row['name'],
        'source': row['source']
    }
    
    query = """
    match (grantee:Grantee:Entity {name: $name}) 
    set grantee.source = $source
    """
    execute_query(query, params)
    
    


Ingesting 1 of 22 records...
Ingesting 2 of 22 records...
Ingesting 3 of 22 records...
Ingesting 4 of 22 records...
Ingesting 5 of 22 records...
Ingesting 6 of 22 records...
Ingesting 7 of 22 records...
Ingesting 8 of 22 records...
Ingesting 9 of 22 records...
Ingesting 10 of 22 records...
Ingesting 11 of 22 records...
Ingesting 12 of 22 records...
Ingesting 13 of 22 records...
Ingesting 14 of 22 records...
Ingesting 15 of 22 records...
Ingesting 16 of 22 records...
Ingesting 17 of 22 records...
Ingesting 18 of 22 records...
Ingesting 19 of 22 records...
Ingesting 20 of 22 records...
Ingesting 21 of 22 records...
Ingesting 22 of 22 records...


# Ingest grants

### `grants.csv` 
Link grantees to grant metadata. Grantees may have multiple grants from different grant initiatives

```(g:Grantee)<-[r:GRANTEE]-(gr:Grant:GrantInitiative: {metadata..})-[:FUNDED]-(e:Funder)```


**Grant Initiative**

### Todos
- Find document announcing/outlining parameters for each grant initiative.
  
```
{
    "name": *name of grant program or, if direct, grant announcement*,
    "grantPlatform": *if applicable, platform used to administer/fund grant*,
    "tokenAddress": token contract address
} 
```

**GRANTEE relationsip**

```
{
    "grantApprovalAction": *Action to approve grant, https://snapshot.org/#/arbitrumfoundation.eth/proposal/...*
    "grantSubmissionUrl": *Grantee published document outlining work done for grant*,
    "fundingAuthAction": *Action to distribute funds, multisig tx (coming soon), i.e. tally.xyz/gov/arbitrum/proposal*...,
    "percVoteWon": *if available, percentage of voters won*,
    "tokenAddress": *token grant paid out in*,
    "amount": *count of token granted*,
}
```

**Funder**

More metadata to come...

```
{
    "name": i.e. Arbitrum Foundation
}
```


In [25]:
grants_df = pd.read_csv('inputs/grants_initial.csv')
print(f"There are {len(grants_df)} grants to ingest")
grants_df.head(5)

There are 139 grants to ingest


,A,name,Funding Platform,Amount,Grant Approval Vote,Grant Approval Type,Funding Authorization Vote,Voting Plurality,Submission,Funder,Grant Initiative,Grant Token Address,Count Voters
0,NaN,WOOFi,Tally,1000000,https://snapshot.org/#/arbitrumfoundation.eth/...,Snapshot,https://www.tally.xyz/gov/arbitrum/proposal/13...,82.61%,https://forum.arbitrum.foundation/t/woofi-fina...,Arbitrum Foundation,STIP - Round 1,0x912ce59144191c1204e64559fe8253a0e49e6548,NaN
1,NaN,Gains Network,Tally,7000000,https://snapshot.org/#/arbitrumfoundation.eth/...,NaN,https://www.tally.xyz/gov/arbitrum/proposal/13...,93.92%,https://forum.arbitrum.foundation/t/gains-netw...,Arbitrum Foundation,STIP - Round 1,0x912ce59144191c1204e64559fe8253a0e49e6548,NaN
2,NaN,DefiEdge,Tally,200000,https://snapshot.org/#/arbitrumfoundation.eth/...,NaN,https://www.tally.xyz/gov/arbitrum/proposal/13...,99.38%,https://forum.arbitrum.foundation/t/defiedge-f...,Arbitrum Foundation,STIP - Round 1,0x912ce59144191c1204e64559fe8253a0e49e6548,NaN
3,NaN,Synapse,Tally,2000000,https://snapshot.org/#/arbitrumfoundation.eth/...,NaN,https://www.tally.xyz/gov/arbitrum/proposal/13...,72.32%,https://snapshot.org/#/arbitrumfoundation.eth/...,Arbitrum Foundation,STIP - Round 1,0x912ce59144191c1204e64559fe8253a0e49e6548,NaN
4,NaN,PancakeSwap,Tally,200000,https://snapshot.org/#/arbitrumfoundation.eth/...,NaN,https://www.tally.xyz/gov/arbitrum/proposal/13...,88.37%,https://forum.arbitrum.foundation/t/pancakeswa...,Arbitrum Foundation,STIP - Round 1,0x912ce59144191c1204e64559fe8253a0e49e6548,NaN


In [26]:
### create grant/grant initiatives

grant_initiatives_df = grants_df[['Grant Initiative', 'Funding Platform', 'Grant Token Address']]
unique_grant_initiatives_df = grant_initiatives_df.drop_duplicates()
counter = 0 
total = len(unique_grant_initiatives_df)
print(f"Ingesting {str(total)} grants...")

for index, row in unique_grant_initiatives_df.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} out of {str(total)} records...")

    params = {
        'name': row['Grant Initiative'],
        'grantPlatform': row['Funding Platform'],
        'tokenAddress': row['Grant Token Address']
    }

    query = """
    merge (grant:GrantInitiative:Grant {name: $name})
    set grant.grantPlatform = $grantPlatform
    set grant.tokenAddress = $tokenAddress
    """
    execute_query(query, params)
    
    


Ingesting 11 grants...
Ingesting 1 out of 11 records...
Ingesting 2 out of 11 records...
Ingesting 3 out of 11 records...
Ingesting 4 out of 11 records...
Ingesting 5 out of 11 records...
Ingesting 6 out of 11 records...
Ingesting 7 out of 11 records...
Ingesting 8 out of 11 records...
Ingesting 9 out of 11 records...
Ingesting 10 out of 11 records...
Ingesting 11 out of 11 records...


In [27]:
### connect grantees
counter = 0 
total = len(grants_df)
print(f"Ingesting {str(total)} grantee to grant rels...")

for index, row in grants_df.iterrows():
    counter += 1
    print(f"Ingesting {str(counter)} out of {str(total)} records...")

    params = {
        'name': row['name'],
        'grantInitiative': row['Grant Initiative'],
        'grantApprovalAction': row['Grant Approval Vote'],
        'grantSubmission': row['Submission'],
        'fundingAuthAction': row['Funding Authorization Vote'],
        'percVoteWon': row['Voting Plurality'],
        'tokenAddress': row['Grant Token Address'],
        'amount': row['Amount']
    }

    query = """
    match (grant:GrantInitiative:Grant {name: $grantInitiative})
    match (grantee:Grantee:Entity {name: $name})
    with grant, grantee
    merge (grant)-[r:GRANTEE]->(grantee)
    set r.amount = $amount
    set r.tokenAddress = $tokenAddress
    set r.percVoteWon = $percVoteWon
    set r.grantSubmission = $grantSubmission
    set r.grantApprovalAction = $grantApprovalAction
    """
    execute_query(query, params)



Ingesting 139 grantee to grant rels...
Ingesting 1 out of 139 records...
Ingesting 2 out of 139 records...
Ingesting 3 out of 139 records...
Ingesting 4 out of 139 records...
Ingesting 5 out of 139 records...
Ingesting 6 out of 139 records...
Ingesting 7 out of 139 records...
Ingesting 8 out of 139 records...
Ingesting 9 out of 139 records...
Ingesting 10 out of 139 records...
Ingesting 11 out of 139 records...
Ingesting 12 out of 139 records...
Ingesting 13 out of 139 records...
Ingesting 14 out of 139 records...
Ingesting 15 out of 139 records...
Ingesting 16 out of 139 records...
Ingesting 17 out of 139 records...
Ingesting 18 out of 139 records...
Ingesting 19 out of 139 records...
Ingesting 20 out of 139 records...
Ingesting 21 out of 139 records...
Ingesting 22 out of 139 records...
Ingesting 23 out of 139 records...
Ingesting 24 out of 139 records...
Ingesting 25 out of 139 records...
Ingesting 26 out of 139 records...
Ingesting 27 out of 139 records...
Ingesting 28 out of 139 r

In [28]:
## connect funders

daos =  ['Arbitrum Foundation', 'Uniswap', 'Allo Protocol', 'Gitcoin', ]
companies = ['Questbook', 'Buidlbox']

In [29]:
for i in daos:
    execute_query(f"""merge (dao:Dao:Entity {{name: '{i}'}})""")

In [30]:
for i in companies:
    execute_query(f"""merge (company:Company:Entity {{name: '{i}'}})""")

In [31]:
execute_query("""match (n:Entity {name:"Arbitrum Foundation"}) 
match (grant:Grant) where grant.grantPlatform = 'Tally'
with grant, n
merge (n)-[r:FUNDED]->(grant)
return count(r)""")

,count(r)
0,1


In [32]:
execute_query("""match (n:Entity {name:"Uniswap"}) 
match (grant:Grant) where grant.grantPlatform = 'Gitcoin'
with grant, n
merge (n)-[r:FUNDED]->(grant)
return count(r)""")

,count(r)
0,3


In [33]:
execute_query("""match (n:Entity {name:"Arbitrum Foundation"}) 
match (grant:Grant) where grant.grantPlatform = 'Gitcoin'
with grant, n
merge (n)-[r:FUNDED]->(grant)
return count(r)""")

,count(r)
0,3


In [34]:
execute_query("""match (n:Entity {name:"Arbitrum Foundation"}) 
match (grant:Grant) where grant.grantPlatform = 'Gitcoin'
with grant, n
merge (n)-[r:FUNDED]->(grant)
return count(r)""")

,count(r)
0,3


In [35]:
execute_query("""match (n:Entity {name:"Arbitrum Foundation"}) 
match (grant:Grant) where grant.grantPlatform = 'Questbook'
with grant, n
merge (n)-[r:FUNDED]->(grant)
return count(r)""")

,count(r)
0,3


In [36]:
execute_query("""match (n:Entity {name:"Questbook"}) 
match (grant:Grant) where grant.grantPlatform = 'Questbook'
with grant, n
merge (n)-[r:FUNDED]->(grant)
return count(r)""")

,count(r)
0,6


In [37]:
execute_query("""match (n:Entity {name:"Gitcoin"}) 
match (grant:Grant) where grant.grantPlatform = 'Questbook'
with grant, n
merge (n)-[r:FUNDED]->(grant)
return count(r)""")

,count(r)
0,3


In [40]:
execute_query("""
match (n:Grantee)-[]-(grant:Grant)
where n.name in ['DAO Driven Crowdfunding', 'Spray', 'Convction Voting App',
'Build3r Labs', 'AGDM', 'Magari', 'SolidGrant', 'Alloiance', 'Buidl3r', 'Allo Explorer']
with grant
match (x:Entity)
where x.name in ['Arbitrum Foundation', 'Allo Protocol', 'Gitcoin', 'Buidlbox']
MERGE (x)-[r:FUNDED]->(grant)""")



""
